# งานที่ 1b: สร้างข้อความโดยใช้พรอมต์คำสั่งที่มีบริบท

ในโน้ตบุ๊กนี้ คุณจะได้เรียนรู้วิธีสร้างการตอบกลับทางอีเมลให้กับลูกค้าที่ไม่พอใจกับคุณภาพการบริการลูกค้าที่ได้รับจากวิศวกรสนับสนุนลูกค้า คุณจะให้บริบทเพิ่มเติมให้กับโมเดลโดยรวมเนื้อหาของอีเมลจริงที่ได้รับจากลูกค้าที่ไม่พอใจ

คุณจะเพิ่มความซับซ้อนมากขึ้นด้วยความช่วยเหลือของ PromptTemplates เพื่อใช้ประโยชน์จากเฟรมเวิร์ก LangChain สำหรับกรณีการใช้งานที่คล้ายกัน PromptTemplates จะช่วยให้คุณสามารถสร้างเชลล์ทั่วไปซึ่งสามารถเติมข้อมูลในภายหลังและรับเอาต์พุตโมเดลตามสถานการณ์ที่แตกต่างกัน

[LangChain](https://python.langchain.com/docs/get_started/introduction.html) เป็นเฟรมเวิร์กสำหรับการพัฒนาแอปพลิเคชันที่ขับเคลื่อนโดยโมเดลภาษา แง่มุมสำคัญของเฟรมเวิร์กนี้ช่วยให้เราสามารถขยายโมเดลภาษาขนาดใหญ่ได้โดยการเชื่อมองค์ประกอบต่างๆ เข้าด้วยกันเพื่อสร้างกรณีการใช้งานขั้นสูง

เนื่องจากบริบทเพิ่มเติมในพรอมต์คำสั่ง เนื้อหาที่ผลิตในโน้ตบุ๊กนี้มีคุณภาพและความเกี่ยวข้องที่ดีกว่าเนื้อหาที่ผลิตผ่านพรอมต์คำสั่งแบบ zero-shot ก่อนหน้านี้ พรอมต์คำสั่งที่ใช้ในโน้ตบุ๊กนี้จะสร้างเทมเพลตพรอมต์คำสั่ง LangChain ที่กำหนดเองสำหรับการเพิ่มบริบทให้กับคำขอสร้างข้อความ

#### สถานการณ์
คุณคือ Bob ผู้จัดการฝ่ายบริการลูกค้าของ AnyCompany และลูกค้าบางคนไม่พอใจกับการบริการลูกค้าและกำลังให้ข้อเสนอแนะเชิงลบเกี่ยวกับบริการที่ให้โดยวิศวกรฝ่ายสนับสนุนลูกค้า ตอนนี้ คุณอยากตอบกลับลูกค้าเหล่านั้นอย่างถ่อมตนเพื่อขอโทษสำหรับบริการที่ไม่ดีและเพื่อเรียกความไว้วางใจกลับคืนมา คุณต้องการความช่วยเหลือจากโมเดลภาษาขนาดใหญ่ (LLM) เพื่อสร้างอีเมลจำนวนมากให้แก่คุณสำหรับอีเมลที่ให้ความเป็นมิตรอย่างเป็นธรรมชาติและปรับให้เหมาะกับความรู้สึกของลูกค้าจากการติดต่อทางอีเมลก่อนหน้านี้

ในสถานการณ์นี้ คุณสามารถใช้ประโยชน์จากพลังของ PromptTemplates ของ LangChain เพื่อสร้างเชลล์ทั่วไปสำหรับการสร้างการตอบกลับอีเมลส่วนบุคคลโดยอิงตามอีเมลก่อนหน้าของลูกค้า PromptTemplate จะใช้เนื้อหาจากอีเมลต้นฉบับของลูกค้า ทำให้ LLM เข้าใจบริบทและความรู้สึก จากนั้นจะสร้างการตอบกลับที่เกี่ยวข้องและกำหนดเอง

## งานที่ 1b.1: การตั้งค่าสภาพแวดล้อม

ในงานนี้ ให้ตั้งค่าสภาพแวดล้อมของคุณ

In [ ]:
#Create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))


bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))

## งานที่ 1b.2: เรียกโมเดล Bedrock LLM

ในงานนี้ ให้คุณสร้างอินสแตนซ์ของคลาส Bedrock จาก llms สิ่งนี้คาดว่าจะเป็น “model_id” ซึ่งเป็น Amazon Resource Name (ARN) ของโมเดลที่มีอยู่ใน Amazon Bedrock

หรือ คุณสามารถส่งผ่านไคลเอ็นต์ boto3 ที่สร้างไว้ก่อนหน้านี้รวมถึงบาง `model_kwargs` ที่สามารถเก็บพารามิเตอร์ เช่น `temperature `, `top_p`, `max_token_count` หรือ `stop_sequences` ได้ (ข้อมูลเพิ่มเติมเกี่ยวกับพารามิเตอร์สามารถสำรวจได้ในคอนโซล Amazon Bedrock)

โปรดดู [เอกสาร] (https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids-arns.html) สำหรับ ID โมเดลสร้างข้อความที่มีอยู่ภายใต้ Amazon Bedrock

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **หมายเหตุ:** โมเดลที่แตกต่างกันรองรับ `model_kwargs` ที่แตกต่างกัน

In [ ]:
# Model configuration
from langchain_aws import ChatBedrock
from langchain_core.output_parsers import StrOutputParser

model_id = "meta.llama3-8b-instruct-v1:0"
model_kwargs =  { 
        "max_gen_len": 512,
        "temperature": 0,
        "top_p": 1,
}

# LangChain class for chat
chat_model = ChatBedrock(
    client=bedrock_client,
    model_id=model_id,
    model_kwargs=model_kwargs,
)

## งานที่ 1b.3: สร้างเทมเพลตพรอมต์คำสั่งที่กำหนดเอง LangChain

ในงานนี้ ให้คุณสร้างเทมเพลตสำหรับพรอมต์คำสั่งที่คุณสามารถส่งตัวแปรอินพุตที่แตกต่างกันในทุกการรัน สิ่งนี้มีประโยชน์เมื่อคุณต้องสร้างเนื้อหาที่มีตัวแปรอินพุตต่างๆ ที่คุณอาจดึงมาจากฐานข้อมูล

ในงานก่อนหน้านี้ เราเขียนพรอมต์คำสั่งแบบฮาร์ดโค้ด โดยอาจเป็นกรณีที่คุณมีลูกค้าหลายรายส่งข้อเสนอแนะเชิงลบที่คล้ายกัน และตอนนี้คุณต้องการใช้อีเมลของลูกค้าแต่ละรายเพื่อตอบกลับด้วยคำขอโทษ แต่คุณต้องการให้คำตอบกลับเป็นส่วนตัวเล็กน้อย ในเซลล์ต่อไปนี้คุณจะสำรวจวิธีสร้าง `PromptTemplate` เพื่อให้ได้รูปแบบนี้

In [ ]:
# Create a prompt template that has multiple input variables
from langchain.prompts import PromptTemplate

multi_var_prompt = PromptTemplate(
    input_variables=["customerServiceManager", "customerName", "feedbackFromCustomer"], 
    template="""

Human: Create an apology email from the Service Manager {customerServiceManager} at AnyCompany to {customerName} in response to the following feedback that was received from the customer: 
<customer_feedback>
{feedbackFromCustomer}
</customer_feedback>

Assistant:"""
)

# Pass in values to the input variables
prompt = multi_var_prompt.format(customerServiceManager="Bob Smith", 
                                 customerName="John Doe", 
                                 feedbackFromCustomer="""Hello Bob,
     I am very disappointed with the recent experience I had when I called your customer support.
     I was expecting an immediate call back but it took three days for us to get a call back.
     The first suggestion to fix the problem was incorrect. Ultimately the problem was fixed after three days.
     We are very unhappy with the response provided and may consider taking our business elsewhere.
     """
     )


In [ ]:
# get number of tokens
num_tokens = chat_model.get_num_tokens(prompt)
print(f"Our prompt has {num_tokens} tokens")

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **หมายเหตุ:** คุณสามารถเพิกเฉยต่อคำเตือนได้อย่างปลอดภัยและดำเนินการไปยังเซลล์ถัดไป

In [ ]:
#invoke
response = chat_model.invoke(prompt)

In [ ]:
# Configure a Chain to parse output
chain = StrOutputParser()
formatted_response=chain.invoke(response)
print(formatted_response)

คุณได้เรียนรู้สำเร็จว่าการเรียกใช้ LLM โดยไม่มีบริบทใด ๆ อาจไม่ได้ให้เอาต์พุตที่ต้องการ ด้วยการเพิ่มบริบทและใช้เทมเพลตพรอมต์คำสั่งเพื่อจำกัดเอาต์พุตจาก LLM คุณจะสามารถรับเอาต์พุตที่ต้องการได้สำเร็จ

### ลองด้วยตัวเอง
- เปลี่ยนพรอมต์คำสั่งตามฐานการใช้งานเฉพาะของคุณและประเมินเอาต์พุตของโมเดลต่างๆ
- เล่นด้วยความยาวโทเค็นเพื่อทำความเข้าใจเวลาแฝงและการตอบกลับของบริการ
- ใช้หลักการวิศวกรรมการโต้ตอบที่แตกต่างกันเพื่อให้ได้เอาต์พุตที่ดีขึ้น

### เก็บงาน

คุณทำโน้ตบุ๊กนี้เสร็จแล้ว หากต้องการย้ายไปยังส่วนถัดไปของแล็บ ให้ทำดังนี้

- ปิดไฟล์โน้ตบุ๊กนี้
- กลับไปที่เซสชันของแล็บและไปต่อยัง **งานที่ 2**